# Convolutional Neural Network

### Importing the libraries

In [16]:
!pip install --upgrade tensorflow keras Pillow

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: tensorflow in /home/igor/.local/lib/python2.7/site-packages (2.1.0)
Requirement already up-to-date: keras in /home/igor/.local/lib/python2.7/site-packages (2.4.3)
Requirement already up-to-date: Pillow in /home/igor/.local/lib/python2.7/site-packages (6.2.2)


In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [18]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 20,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [21]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [22]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [23]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [24]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [25]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [26]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [27]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))

### Step 5 - Output Layer

In [28]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [29]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [30]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 34s 134ms/step - loss: 0.6866 - accuracy: 0.5477 - val_loss: 0.6195 - val_accuracy: 0.6505
Epoch 2/25
250/250 [==============================] - 33s 132ms/step - loss: 0.6131 - accuracy: 0.6634 - val_loss: 0.5564 - val_accuracy: 0.7130
Epoch 3/25
250/250 [==============================] - 34s 135ms/step - loss: 0.5797 - accuracy: 0.6980 - val_loss: 0.5715 - val_accuracy: 0.6995
Epoch 4/25
250/250 [==============================] - 33s 133ms/step - loss: 0.5578 - accuracy: 0.7086 - val_loss: 0.5230 - val_accuracy: 0.7380
Epoch 5/25
250/250 [==============================] - 33s 133ms/step - loss: 0.5228 - accuracy: 0.7448 - val_loss: 0.4604 - val_accuracy: 0.7755
Epoch 6/25
250/250 [==============================] - 35s 142ms/step - loss: 0.5052 - accuracy: 0.7502 - val_loss: 0.4587 - val_accuracy: 0.7750
Epoch 7/25
250/250 [==============================] - 33s 133ms/step - loss: 0.4810 - accuracy: 0.7653 - val_loss: 0.4441 - val_ac

## Part 4 - Making a single prediction

In [31]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [32]:
print(prediction)

dog


## Fim